# EECS 487 HW 1 Coding (C) Section: Language Model and Naive Bayes Classifier

This notebook contains the coding (C) section of HW 1. In the first problem, you will build two n-gram language models for Yelp reviews, one that operates at the word-level and another that operates on the level of individual characters. You will then use these language models to generate sample Yelp reviews. In the second problem, you will build naive bayes classifiers to distinguish between legitimate news headlines and clickbait.

After this assignment, you will learn to
1. train ngram language models given a text corpus;
2. generate text from a language model;
3. calculate probability of some text given a language model;
4. classify news headlines using naive bayes classifiers;
5. evaluate classifiers by calculating the performance on test set.

**Do not edit anything in this python notebook.** All the code you need to write are in ```language_model.py``` and ```naive_bayes.py```.

## Setup
Run the following cell to upgrade your NLTK version to the latest. You should be using NLTK version >= 3.8 for proper results!!

In [ ]:
!pip install nltk --upgrade  # after running this line once, you can comment this line out
import nltk
print(nltk.__version__) # this should print out 3.8.1 if you have installed the latest version of NLTK properly

Before we get started, run the following cell to load the autoreload extension so that functions in ```language_model.py``` and ```naive_bayes.py``` will be re-imported into the notebook every time we run them. We also need to import all necessary packages.

In [ ]:
%load_ext autoreload
%autoreload 2

import pickle

import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

from language_model import *
from naive_bayes import *

## C.1 N-gram Language Model [40 points]
In this problem, you will train two language models on Yelp reviews, one word-level (up to trigrams) and the other character-level (up to 4-grams). The [dataset provided](https://www.kaggle.com/datasets/omkarsabnis/yelp-reviews-dataset?resource=download) is a subset of [a larger dataset](https://www.yelp.com/dataset) published by Yelp. We downloaded the file ```yelp.csv``` for you. To begin, you need to first load the data. Here we provide the code for you, but take a look at how we do it because you will need to load the data by yourself later.

In [ ]:
filename = 'yelp.csv'
df = pd.read_csv(filename)

all_text = df['text']

trn_text, dev_text = train_test_split(all_text, test_size=0.2, random_state=42)
trn_text, dev_text = trn_text.reset_index(drop=True), dev_text.reset_index(drop=True)
print("trn_text:")
print(trn_text)
print("\ndev_text:")
print(dev_text)

### C.1.1 Data processing and n-gram counts [10 points]
Now you need to train your language models on these reviews. You need to implement the class ```NGramLM```. First, **fill in** the function ```get_ngram_counts``` to process the reviews and get the counts of all n-grams in the reviews.

Program your `NGramLM` class and `get_ngram_counts` function to collect n-grams of size `n <= ngram_size`. If you refer to the cells below, you will see that `ngram_size = 3` for the word-level model (so collect unigrams, bigrams, and trigrams) and `ngram_size = 4` for the character-level model (so collect unigrams, bigrams, trigrams, and four-grams).

Store these n-grams in the dictionary `self.ngram_count`. This dictionary will contain other dictionaries as values. For instance, `self.ngram_count[0]` will be a dictionary containing all of the unigrams, and `self.ngram_count[1]` will be a dictionary containing all the bigrams. To access the count of a unigram, simply use it as a key in the unigram dictionary: ```self.ngram_count[0]["word1"]``` will be $C(word1)$. To access the count of a bigram (or trigram or four-gram), simply use a tuple: ```self.ngram_count[1][("word1", "word2")]``` will be $C(word1, word2)$.

Use the following rules when processing the reviews:
- Prepend **two/three** &lt;s&gt; at the beginning of each review as BOS tokens (two for trigram model and three for four-gram model), and append one &lt;/s&gt; at the end of a review as the EOS token.
- Convert all letters to lowercase.
- Tokenize each review. Use `nltk.tokenize.word_tokenize` for the word-level model and `char_tokenizer` (defined below) for the character-level model. For your convenience, each is passed into NGramLM, so you can program the internals of NGramLM to simply utilize whichever tokenizer is passed in. Do not split BOS and EOS tokens (i.e., even in the character-level model, `<s>` will be a single token rather than three tokens).
- Replace all unigrams that occur < 2 times with "UNK". For the character-level model, this means that all characters occuring < 2 times should be replaced with UNK. **NOTE:** do this before collecting (n>1)-grams. For instance, if the word "apple" appeared < 2 times, then the bigram ("red", "apple") should instead be ("red", "UNK").
- You will need to make the function ```get_ngram_counts``` flexible enough so that (1) it can operate on both the character-level and the word-level and (2) it can operate on a variety of maximum ngram sizes (as determined by `ngram_size`)
- Do **NOT** remove punctuation.

Hint: ```collections.defaultdict``` is useful.

In [ ]:
bos_token, eos_token = '<s>', '</s>'
ngram_size = 3 # Use trigrams
word_lm = NGramLM(bos_token, eos_token, word_tokenize, ngram_size)
word_lm.get_ngram_counts(trn_text.tolist())
print(f"Number of unigrams: {len(word_lm.ngram_count[0])}")
least_unigram = min(word_lm.ngram_count[0].keys(), key=lambda x: word_lm.ngram_count[0][x])
print(f"Unigram with smallest count: {least_unigram}\tCount: {word_lm.ngram_count[0][least_unigram]}")
print(f"Unknown unigram: {word_lm.ngram_count[0]['UNK']}")
print(f"Number of BOS token: {word_lm.ngram_count[0][bos_token]}")
print(f"Number of bigrams: {len(word_lm.ngram_count[1])}")

Now that you have tested ```get_ngram_counts``` on the word-level model (above), test it on the character-level model below.

In [ ]:
def char_tokenizer(text):
    return [char for char in text]

ngram_size = 4 # Use four-grams
char_lm = NGramLM(bos_token, eos_token, char_tokenizer, ngram_size)
char_lm.get_ngram_counts(trn_text.tolist())
print(f"Number of unigrams: {len(char_lm.ngram_count[0])}")
least_unigram = min(char_lm.ngram_count[0].keys(), key=lambda x: char_lm.ngram_count[0][x])
print(f"Unigram with smallest count: {least_unigram}\tCount: {char_lm.ngram_count[0][least_unigram]}")
print(f"Unknown unigram: {char_lm.ngram_count[0]['UNK']}")
print(f"Number of BOS token: {char_lm.ngram_count[0][bos_token]}")
print(f"Number of bigrams: {len(char_lm.ngram_count[1])}")

### C.1.2 Add-k Smoothing [5 points]
As discussed in the lecture, simply counting the number of occurrence of n-grams will assign 0 probability to n-grams that don't appear in the training corpus and thus cannot generalize to unseen data. To mitigate this, you need to implement some smoothing techniques. **Fill in** the function ```add_k_prob``` so that given a bigram $(w_1, w_2)$, a unigram $w_3$, and $k$, return $p(w_3|(w_1, w_2))$ after applying add-k smoothing.<br>

Notes:
- Program this flexibly enough so that, for the character-level model, the model can smooth over trigrams and unigrams rather than bigrams and unigrams.
- &lt;s&gt; should **NOT** be considered when calculating the vocabulary size because it will never be generated by the language model (although it's in ```self.unigram_count```). &lt;/s&gt; and "UNK" should both be treated as tokens in the vocabulary.

In [ ]:
# prepare counts
test_data = ['An apple', 'A circle', 'This dot', 'That triangle', 'The red apple', 'The red circle', 'The blue dot', 'The blue triangle']
k = 0.5
test_word_lm = NGramLM(bos_token, eos_token, word_tokenize, 3)
test_word_lm.get_ngram_counts(test_data)
print(f"Vocabulary: {list(test_word_lm.ngram_count[0].keys())}")
# test
bigram = ('the', 'red')
unigram = 'apple'
print(f"Probability of seen: {test_word_lm.add_k_smooth_prob(bigram, unigram, k)}")

bigram = ('the', 'blue')
unigram = 'apple'
print(f"Probability of unseen: {test_word_lm.add_k_smooth_prob(bigram, unigram, k)}")

Now test out the same smoothing code on the character-level model.

In [ ]:
test_data = ['plan', 'plant', 'planet']
k = 0.5
test_char_lm = NGramLM(bos_token, eos_token, char_tokenizer, 4)
test_char_lm.get_ngram_counts(test_data)
print(f"Vocabulary: {list(test_char_lm.ngram_count[0].keys())}")
# test
trigram = ('p', 'l', 'a')
unigram = 'n'
print(f"Probability of seen: {test_char_lm.add_k_smooth_prob(trigram, unigram, k)}")

trigram = ('p', 'l', 'a')
unigram = 't'
print(f"Probability of unseen: {test_char_lm.add_k_smooth_prob(trigram, unigram, k)}")

### C.1.3 Linear interpolation [4 points]
Similarly, **fill in** the function ```linear_interp_prob``` so that given a bigram $(w_1, w_2)$, a unigram $w_3$, and list of values [$\lambda_1$, $\lambda_2$, $\lambda_3$], return $p(w_3|(w_1, w_2))$ after applying linear interpolation.

Once again, implement this flexibly enough to operate on four-grams for the character-level model.

A couple notes:
- In certain instances, you may end up with a fraction 0/0 due to the fact that the count for both the n-gram and (n-1)-gram are equal to zero. In these instances, treat the entire fraction as 0 (even though technically $\frac{0}{0} \neq 0$).
- Recall from the last sub-problem that `<s>` should not be included when calculating the vocabulary size.

In [ ]:
lambda1 = 0.6
lambda2 = 0.2
lambda3 = 0.2
lambdas = [lambda1, lambda2, lambda3]

bigram = ('the', 'red')
unigram = 'apple'
print(f"Probability of seen: {test_word_lm.linear_interp_prob(bigram, unigram, lambdas)}")

bigram = ('the', 'blue')
unigram = 'apple'
print(f"Probability of unseen: {test_word_lm.linear_interp_prob(bigram, unigram, lambdas)}")

Now test ```linear_interp_prob``` on the character-level model.

In [ ]:
lambda1 = 0.6
lambda2 = 0.2
lambda3 = 0.1
lambda4 = 0.1
lambdas = [lambda1, lambda2, lambda3, lambda4]

trigram = ('p', 'l', 'a')
unigram = 'n'
print(f"Probability of seen: {test_char_lm.linear_interp_prob(trigram, unigram, lambdas)}")

trigram = ('p', 'l', 'a')
unigram = 't'
print(f"Probability of unseen: {test_char_lm.linear_interp_prob(trigram, unigram, lambdas)}")

### C.1.4 Calculate next word probability [2 points]
**Fill in** the function ```get_probability``` that calculates $p(w_3|(w_1, w_2))$ using either add-k smoothing or linear interpolation that you implemented above. The input is a dictionary that specifies how should you do the smoothing. Once again, program this function flexibly so that it works in the character-level model as well.

In [ ]:
smoothing_args1 = {
    'method': 'add_k',
    'k': 0.5
}
smoothing_args2 = {
    'method': 'linear',
    'lambdas': [0.6, 0.2, 0.2]
}
bigram = ('a', 'red')
unigram = 'dot'
print(f"Add-k smoothing: {test_word_lm.get_probability(bigram, unigram, smoothing_args1)}")
print(f"Linear interpolation: {test_word_lm.get_probability(bigram, unigram, smoothing_args2)}")

### C.1.5 Calculate perplexity [4 points]
One way to evaluate the language model is to calculate its perplexity on some validation data. **Fill in** the function ```get_perplexity``` so that given a document and smoothing arguments, return the perplexity of the document. Remember to follow the **same** processing steps you used previously. To avoid underflow issue, remember to calculate the log of perplexity first. i.e., $PPL(W)=\exp\left(\log\left(\sqrt[N]{\prod_{i=1}^N{\frac{1}{p(w_i|w_{i-2}w_{i-1})}}}\right)\right)=\exp\left(-\frac{1}{N}\sum_{i=1}^N{\log{p(w_i|w_{i-2}w_{i-1})}}\right)$, where $w_0=w_{-1}=$ &lt;s&gt; and $w_N=$ &lt;/s&gt;. Also, once again, remember to program this flexibly enough that it can work for both the word-level and character-level models.

In [ ]:
text = "This sentence contains unseen words."

for word in text.split():
    if word in test_word_lm.ngram_count[0].keys():
        count = test_word_lm.ngram_count[0][word]
    else:
        count = 0
    print(f"Count of {word} is {count}")

print(f"Add-k smoothing: {test_word_lm.get_perplexity(text, smoothing_args1)}")
print(f"Linear interpolation: {test_word_lm.get_perplexity(text, smoothing_args2)}")

### C.1.6 Search hyperparameters [6 points]
Now you are ready to find your best language models! First find the best k value for the word-level model and for the character-level model using add-k smoothing. You need to search k in this list: \[0.2, 0.4, 0.6, 0.8, 1.0\].

**Fill in** the function ```search_k``` such that given a validation set, return the best k value on it. Print out the perplexity (average on the whole validation set) for each k.

In [ ]:
print("Word LM")
word_k = word_lm.search_k(dev_text)
print(f"Best k: {word_k}")
print("Char LM")
char_k = char_lm.search_k(dev_text)
print(f"Best k: {char_k}")

Similarly, **fill in** the function ```search_lambda``` such that, given a validation set, returns the best $\lambda$ values on it. You need to choose the search list by yourself. Print out the best set of $\lambda$ and corresponding perplexity. To get full credits, your perplexity scores need to be < 180 for the word-level model and < 15 for the character-level model.

Note: this code block might take a couple minutes to run.

In [ ]:
print("Word LM")
word_lambda = word_lm.search_lambda(dev_text)
print("Char LM")
char_lambda = char_lm.search_lambda(dev_text)

### C.1.7 Generate reviews [5 points]
Finally, you can automatically generate text using your language models. **Fill in** the function ```generate_text``` to generate a sentence based on an input prompt. To generate the text, you need to find the distribution of the next word given previous two words (or next character given the previous three characters). Then you need to **randomly** sample the next word/character based on the distribution (i.e., do **NOT** use greedy decoding, which is deterministically choosing the most probable unigram). After the word/character is sampled, append it to the current text and continue generating the next word/character. You need to repeat this process untill the current sequence **has 15 words/characters** (including prompts) or you **generate the &lt;/s&gt; token**.

Note that there exist more advanced methods to generate text from language models such as beam search, top-k sampling, and top-p sampling. You can refer to [this blog](https://huggingface.co/blog/how-to-generate) to get an idea of what they are. In this assignment, you are not required to implement the advanced methods. Simply sampling from the trigram/four-gram distribution is good enough.

Begin with the word-level model here:

In [ ]:
prompts = [['The', 'location'], ['We', 'ate'], ['I', 'thought'], ['It', 'had']]
for prompt in prompts:
    print("Word LM")
    word_lm.generate_text(prompt, smoothing_args1)

Now test the character-level model:

In [ ]:
prompts = ['The store', 'We ate']
for prompt in prompts:
    print("Char LM")
    prompt_tokenized = char_tokenizer(prompt)
    char_lm.generate_text(prompt_tokenized, smoothing_args1)

### C.1.8 Predict the class of a review [4 points]
The Yelp review dataset contains several columns specifying attributes of each review:
- ```stars``` indicates on a scale from 1 to 5 how many stars the reviewer gave in the review
- ```cool```, ```useful```, and ```funny``` all indicate how many users rated the review as cool, useful or funny

Using the same techniques developed above, this section requires you to do the following:
- Fill in the function ```load_new_data``` to generate a split in the data. For instance, one class could be reviews with at least one funny rating and the opposing class could be reviews not marked as funny by anyone. In this function, use the techniques at the beginning of the notebook to split the overall dataset into two classes of your choice
- Fill in the function ```predict_class``` to predict the class to which the review in ```test_review.txt``` belongs. This function will take in two word-level language models (each trained on the training data for one class) and compute the probability of the review generated by each language model. I.e., $p(W|LM_1)$ and $p(W|LM_2)$, or equivalently $PPL(W)$ based on $LM_1$ and $LM_2$. Print out the perplexity of each language model and your prediction.

The test review used has the following attributes:
- ```stars``` : 2
- ```useful``` : 3
- ```funny``` : 7
- ```cool``` : 3

In [ ]:
filename = 'yelp.csv'
df = pd.read_csv(filename)

smoothing_args = {
    'method': 'linear',
    'lambdas': word_lambda
}

class1_data, class2_data = load_new_data(df)

class1_lm = NGramLM(bos_token, eos_token, word_tokenize, 3)
class1_lm.get_ngram_counts(class1_data)

class2_lm = NGramLM(bos_token, eos_token, word_tokenize, 3)
class2_lm.get_ngram_counts(class2_data)

predict_class('test_review.txt', class1_lm, class2_lm, smoothing_args)

## C.2 Naive Bayes for Text Classification [26 points]
In this problem, you will build naive bayes classifiers to do text classification. You will use the clickbait headlines dataset, which contains examples of legitimate news headlines and clickbait news headlines. The original dataset can be found in [this GitHub repository](https://github.com/bhargaviparanjape/clickbait) and [this paper](https://arxiv.org/abs/1610.09786).
### C.2.1 Load dataset [4 points]
To get started, **fill in** the function ```load_headlines``` to load the clickbait dataset into pandas dataframes. The file ```clickbait_data.csv``` contains a partially processed subset of the data. It contains two columns: (1) ```is_clickbait``` is 1 when the row contains a clickbait headline and 0 when it doesn't and (2) ```text```, which contains the headline itself.

To get started, **fill in** the function ```load_headlines``` to load the clickbait dataset into a pandas dataframe. To do this, you will need to do the following:

1. Read in the ```text``` and ```is_clickbait``` columns.
2. Rename the ```is_clickbait``` column to ```label```

In [ ]:
from naive_bayes_answer import *

all_data = load_headlines('clickbait_data.csv')

(train, test) = train_test_split(all_data, train_size=0.9)

display(train)
display(test)

### C.2.2 Dataset statistics [3 points]
Before start training classifiers, you need to calculate some basic statistics of the dataset. **Fill in** the function ```get_basic_stats``` to print out the following statistics of the training data:
- Average number of tokens per headline
- Standard deviation of the number of tokens per headline
- Total number of legitimate headlines
- Total number of clickbait headlines

Note: you can use any tokenization method you like.

In [ ]:
get_basic_stats(train)

### C.2.3 Data processing and ngram calculation [6 points]
Now you need to calculate the ngram counts. **Fill in** the function ```fit``` that, given a dataframe of training data, calculates the ngram counts in each category and the prior probability for each category. Concretely, **store** the total occurrence of each ngram in each category in a list called ```self.ngram_count``` so that ```self.ngram_count[0]``` contains $count(w, c_0)$ for all $w$ in the vocabulary, and ```self.ngram_count[1]``` contains $count(w, c_1)$, etc. ```self.ngram_count[i]``` should be an array of shape $(1,|V|)$, where $V$ is the vocabulary (total vocabulary across both classes). **Store** the total occurrence of all ngrams in each category in a list called ```self.total_count``` so that ```self.total_count[0]``` $=\sum_{w\in V}count(w, c_0)$, and ```self.total_count[1]``` $=\sum_{w\in V}count(w, c_1)$, etc. **Store** the prior probability for each category in ```self.category_prob```. You need to follow these rules when calculating the counts:
- convert all letters to lowercase;
- include both unigrams and bigrams;
- ignore terms that appear in more than 80\% of the headlines;
- ignore terms that appear in less than 3 headlines.

Hint: use ```CountVectorizer``` in sklearn and store it as ```self.vectorizer```. You need to use **both legitimate and clickbait headlines** to get the vocabulary.

In [ ]:
naive_bayes = NaiveBayes()
naive_bayes.fit(train)
print(f"Probability for each category: {naive_bayes.category_prob}")
print(f"Length of self.ngram_count: {len(naive_bayes.ngram_count)}")
print(f"Shape of the counts for 1st category: {naive_bayes.ngram_count[0].shape}")
print(f"Number of non-zero terms for 1st category: {(naive_bayes.ngram_count[0] > 0).sum()}")
print(f"Maximum count of the 1st category: {naive_bayes.ngram_count[0].max()}")
print(f"Minimum count of the 1st category: {naive_bayes.ngram_count[0].min()}")
print(f"Sum of ngram count for 1st category: {naive_bayes.ngram_count[0].sum()}")
print(f"Total count for each category: {naive_bayes.total_count}")

### C.2.4 Calculate posterior probability for a category [4 points]
Next, you will use the vectorizer and ngram counts to calculate the posterior probability of a category. In this homework, we have two categories: legitimate and clickbait. **Fill in** the function ```calculate_prob``` that given a list of articles $docs$, a category index $i$, return $\log\left(p(c_i)p(d|c_i)\right)=\log\left(p(c_i)\prod_{x\in X}p(x|c_i)\right)$ for each article $d$ in $docs$, where $X$ is the set of unigrams and bigrams in **both** article $d$ and vocabulary $V$.

- Use **add-one smoothing** in your calculation.
- Simply discard unseen unigrams/bigrams (do not use add-one smoothing to account for them).
- Calculate the **sum of logarithms** to avoid issues with underflow.

In [ ]:
test_docs = ["United Kingdom officially exits the European Union",
 "How to Lose a Guy in 10 Days"]
prob1 = naive_bayes.calculate_prob(test_docs, 0)
prob2 = naive_bayes.calculate_prob(test_docs, 1)
print(f"Probability for category 0: {prob1}")
print(f"Probability for category 1: {prob2}")

### C.2.5 Predict labels for new headlines [2 points]
With the posterior probability of each category, you can predict the label for new headlines. **Fill in** the function ```predict``` that, given a list of headlines, returns the predicted categories of the headlines.

In [ ]:
preds = naive_bayes.predict(test_docs)
print(f"Prediction: {preds}")

### C.2.6 Calculate evaluation metrics [5 points]
To evaluate a classifier, you need to calculate some evaluation metrics. **Fill in** the function ```evaluate``` that, given a list of predictions and a list of true labels, returns the accuracy, macro f1-score, and micro f1-score. You can **NOT** use functions in sklearn.

In [ ]:
predictions = [1,1,0,1,0,0,1]
labels = [1,0,0,1,0,1,1]
accuracy, mac_f1, mic_f1 = evaluate(predictions, labels)
print(f"Accuracy: {accuracy}")
print(f"Macro f1: {mac_f1}")
print(f"Micro f1: {mic_f1}")

### C.2.7 Test classifier on test data [2 points]
Finally, you are ready to evaluate your classifier on the test data! Run the following cell to make predictions and print out performance.

In [ ]:
predictions = naive_bayes.predict(test.text.tolist())
labels = test.label.tolist()
accuracy, mac_f1, mic_f1 = evaluate(predictions, labels)
print(f"Accuracy: {accuracy}")
print(f"Macro f1: {mac_f1}")
print(f"Micro f1: {mic_f1}")